In [35]:
import pandas as pd

vehicle_df = pd.read_csv('output_vehicle.csv')
accident_df = pd.read_csv('output_accident.csv')
person_df = pd.read_csv('output_person.csv')

In [36]:
a = accident_df['NO_OF_VEHICLES'].sum()
print("There are %d cars recorded in accident.csv" % a)  # There are 325893 cars recorded in accident.csv

b = vehicle_df.shape[0]
print("There are %d cars(lines) recorded in vehicle.csv" % b)   # There are 325893 cars(lines) recorded in vehicle.csv

print(a==b) # False

There are 325893 cars recorded in accident.csv
There are 259448 cars(lines) recorded in vehicle.csv
False


In [37]:
# First, find the accident_no in the accident table that is not recorded in the vehicle table and delete it

# There is a vehicle record but no corresponding accident can be found
orphan_vehicles = vehicle_df[~vehicle_df['ACCIDENT_NO'].isin(accident_df['ACCIDENT_NO'])]
#print(f"未关联到事故的车辆记录数：{len(orphan_vehicles)}")
print(f"number of accident in vehicle.csv no do not appear in accident.csv: {len(orphan_vehicles)}")  # 0

# There is an accident marked in the accident table, but there is no record in the vehicle table.
orphan_accidents = accident_df[~accident_df['ACCIDENT_NO'].isin(vehicle_df['ACCIDENT_NO'])]
print(f"number of accident in accident.csv not appear in vehicle.csv: {len(orphan_accidents)}")


number of accident in vehicle.csv no do not appear in accident.csv: 0
number of accident in accident.csv not appear in vehicle.csv: 24701


In [38]:
b = accident_df[accident_df['NO_OF_VEHICLES'] == 0].shape[0]
print(f"It exists {b} error terms with vehicles number = 0")   # 5

accident_df = accident_df[accident_df['NO_OF_VEHICLES'] > 0]

# Find all the accident numbers in the vehicle table
vehicle_accident_no = set(vehicle_df['ACCIDENT_NO'])

# Find the accident number in the accident table but not in the vehicle table
invalid_accident_nos = accident_df[~accident_df['ACCIDENT_NO'].isin(vehicle_accident_no)]

print(f"Number of incidents not in the vehicle table: {invalid_accident_nos.shape[0]}")

# Delete these incident records
accident_df = accident_df[accident_df['ACCIDENT_NO'].isin(vehicle_accident_no)]
print(f"Number of rows in the accident table after cleaning: {accident_df.shape[0]}")

It exists 5 error terms with vehicles number = 0
Number of incidents not in the vehicle table: 24696
Number of rows in the accident table after cleaning: 153994


In [39]:
# count the number of vehicles actually corresponding to each accident number
actual_vehicle_counts = vehicle_df['ACCIDENT_NO'].value_counts()

# map the statistical results to the accident table and update the NO_OF_VEHICLES field
accident_df['NO_OF_VEHICLES'] = accident_df['ACCIDENT_NO'].map(actual_vehicle_counts)

VEHICLE_COUNT = accident_df['ACCIDENT_NO'].map(actual_vehicle_counts)

# Find out the inconsistent records for passengers no
notequal = accident_df[accident_df['NO_OF_VEHICLES'] != VEHICLE_COUNT]
print(f"Number of inconsistent records: {len(notequal)}")

Number of inconsistent records: 0


In [40]:
a = accident_df['NO_PERSONS'].sum()
print("There are %d people recorded in accident.csv" % a)  # There are 325893 cars recorded in accident.csv

b = person_df.shape[0]
print("There are %d people(lines) recorded in person.csv" % b)   # There are 325893 cars(lines) recorded in vehicle.csv

print(a==b) # False

There are 383717 people recorded in accident.csv
There are 417616 people(lines) recorded in person.csv
False


In [41]:
# There are person records but no corresponding accidents can be found
orphan_persons = person_df[~person_df['ACCIDENT_NO'].isin(accident_df['ACCIDENT_NO'])]
print(f"number of persons in person.csv not appear in accident.csv: {len(orphan_persons)}")  # 0

# There are incidents marked in the incident table, but there are no records in the person table
orphan_accidents = accident_df[~accident_df['ACCIDENT_NO'].isin(person_df['ACCIDENT_NO'])]
print(f"number of accident in accident.csv not appear in person.csv: {len(orphan_accidents)}")


number of persons in person.csv not appear in accident.csv: 33901
number of accident in accident.csv not appear in person.csv: 0


In [42]:
# delete the rows in the personn table that do not have corresponding accident records
person_df = person_df[person_df['ACCIDENT_NO'].isin(accident_df['ACCIDENT_NO'])]
print(f"The number of rows after invalid records are deleted from the person table:{len(person_df)}")

# have person records but no corresponding accidents can be found
orphan_persons = person_df[~person_df['ACCIDENT_NO'].isin(accident_df['ACCIDENT_NO'])]
print(f"number of persons in person.csv not appear in accident.csv: {len(orphan_persons)}")  # 0

# The incident is marked in the incident table but there is no record in the person table
orphan_accidents = accident_df[~accident_df['ACCIDENT_NO'].isin(person_df['ACCIDENT_NO'])]
print(f"number of accident in accident.csv not appear in person.csv: {len(orphan_accidents)}")

The number of rows after invalid records are deleted from the person table:383715
number of persons in person.csv not appear in accident.csv: 0
number of accident in accident.csv not appear in person.csv: 0


In [43]:
# count the number of people actually corresponding to each accident number
actual_person_counts = person_df['ACCIDENT_NO'].value_counts()

# Map the statistical results to the accident table and update the NO_PERSONS field
accident_df['NO_PERSONS'] = accident_df['ACCIDENT_NO'].map(actual_person_counts)
PERSON_COUNT = accident_df['ACCIDENT_NO'].map(actual_person_counts)

# Find inconsistent records
notequal = accident_df[accident_df['NO_PERSONS'] != PERSON_COUNT]
print(f"Number of inconsistent records: {len(notequal)}")

Number of inconsistent records: 0


### next begin to aggregation to make the accident_no in per csv unique

In [44]:
accident_df.shape[0]  

153994

In [45]:
# Define a general function that aggregates and converts non-zero values ​​to 1
def convert_non_zero_to_one(x):
    return (x > 0).astype(int)

# aggregates operations 
def aggregate_vehicle_data(df):
    agg_dict = {
        'VEHICLE_YEAR_MANUF_h': 'mean',
        'VEHICLE_WEIGHT_h': 'mean'
    }

    # one-hot coding all unique columns that need to be aggregated by max
    binary_cols = [col for col in df.columns if (
        col.startswith('ROAD_SURFACE_TYPE_h_') or
        col.startswith('FUEL_TYPE_') or
        col.startswith('INITIAL_IMPACT_h_') or
        col in ['IS_OVERLOADED_h', 'INTENT_MATCH_h']
    )]

    # aggregation rule
    for col in binary_cols:
        agg_dict[col] = 'max' 

    agg_data = df.groupby('ACCIDENT_NO').agg(agg_dict).reset_index()

    for col in ['VEHICLE_YEAR_MANUF_h', 'VEHICLE_WEIGHT_h']:
        if col in agg_data.columns:
            agg_data[col] = agg_data[col].round().astype('Int64')

    return agg_data


# Aggregation using functions
vehicle_agg_df = aggregate_vehicle_data(vehicle_df)

vehicle_agg_df.to_csv('output_vehicle_agg.csv', index=False)

In [46]:
vehicle_agg_df.shape[0]  

153994

In [47]:
# Then handle the person table

# Fill with the majority AGE_GROUP == 5 (which means unknown)
person_df['AGE_GROUP'] = person_df['AGE_GROUP'].replace(5, pd.NA)
age_mode = person_df['AGE_GROUP'].mode()[0]
person_df['AGE_GROUP'] = person_df['AGE_GROUP'].fillna(age_mode)


# Defining Aggregate Functions
def aggregate_person_data(df):
    agg_dict = {
        'AGE_GROUP': 'mean'
    }

    binary_cols = [col for col in df.columns if (
        col.startswith('SEX_') or
        col.startswith('HELMET_BELT_WORN_risk') or
        col.startswith('ROAD_USER_TYPE_DESC_')
    )]

    for col in binary_cols:
        agg_dict[col] = 'max'

    # aggregation 
    agg_data = df.groupby('ACCIDENT_NO').agg(agg_dict).reset_index()

    # turn to integer
    agg_data['AGE_GROUP'] = agg_data['AGE_GROUP'].round().astype('Int64')

    return agg_data


person_agg_df = aggregate_person_data(person_df)
person_agg_df.to_csv('output_person_agg.csv', index=False)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13400\1232499554.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  person_df['AGE_GROUP'] = person_df['AGE_GROUP'].fillna(age_mode)


In [48]:
person_agg_df.shape[0]  

153994

In [49]:
# handle the accident table
accident_df = accident_df.drop(columns=['SPEED_ZONE_CATEGORY', 'DCA_CODE','ACCIDENT_TYPE_DESC',
                        'DCA_DESC','ROAD_GEOMETRY_DESC','DCA_GROUP'], errors='ignore')

# aggregate function
def aggregate_accident_data(df):
    agg_dict = {
        'TIME_OF_DAY_CODE': 'mean',
        'SEVERITY': 'mean'
    }

    binary_cols = [col for col in df.columns if (
        col.startswith('ACCIDENT_TYPE_') or
        col.startswith('DCA_') or
        col.startswith('LIGHT_CONDITION_') or
        col.startswith('ROAD_GEOMETRY_') or
        col.startswith('SPEED_ZONE_')
    )]

    for col in binary_cols:
        agg_dict[col] = 'max'

    agg_data = df.groupby('ACCIDENT_NO').agg(agg_dict).reset_index()

    if 'SEVERITY' in agg_data.columns:
        agg_data['SEVERITY'] = agg_data['SEVERITY'].round().astype('Int64')

    if 'TIME_OF_DAY_CODE' in agg_data.columns:
        agg_data['TIME_OF_DAY_CODE'] = agg_data['TIME_OF_DAY_CODE'].round().astype('Int64')

    return agg_data


accident_agg_df = aggregate_accident_data(accident_df)
accident_agg_df.to_csv('output_accident_agg.csv', index=False)

In [50]:
accident_agg_df.shape[0]

153994

In [51]:
# Merge the vehicle table (vehicle_agg_df) and the accident table (accident_agg_df)
final_agg_df = pd.merge(accident_agg_df, vehicle_agg_df, on='ACCIDENT_NO', how='left')

# Merge person table (person_agg_df)
final_agg_df = pd.merge(final_agg_df, person_agg_df, on='ACCIDENT_NO', how='left')

final_agg_df.to_csv('final_aggregated_data.csv', index=False)